# Harmonischer Oszillator

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook sollen die Eigenzustände des harmonischen Oszillators numerisch bestimmt werden. Die zugehörige zeitunabhängige Schrödingergleichung lautet in dimensionsloser Form

$$ \left( \frac{1}{2} \frac{\text{d}^2}{\text{d} x^2} + \frac{1}{2} x^2 \right) \psi(x) = E \, \psi(x)\,.$$

Im ersten Teil des Jupyter-Notebooks wird das Problem auf einem diskreten Gitter mit Gitterabstand $\Delta x$ betrachtet, so dass statt der gewöhnlichen Differentialgleichung ein gekoppeltes lineares Gleichungssystem zu lösen ist. Ersetzt man die Ableitung durch einen Differenzenquotienten, ergibt sich für die zweite Ableitung

$$ \frac{\text{d}^2}{\text{d} x^2} \psi(x) \approx \frac{\psi(x-\Delta x) - 2\psi(x) + \psi(x+\Delta x)}{(\Delta x)^2} \,.$$

Mit der diskretisierten Wellenfunktion $\psi_n = \psi(x_n)$ ergibt sich somit das zu lösende lineare Gleichungssystem

$$\frac{1}{(\Delta x)^2}\left(\psi_{n+1}-2\psi_n+\psi_{n-1}\right)+\frac{x_n^2}{2}\psi_n = E\psi_n\,.$$

Im zweiten Teil des Jupyter-Notebooks werden wir ausgehend von geeignet gewählten Ausgangszuständen die niedrigsten drei Eigenzustände durch Energieminimierung bestimmen.

## Importanweisungen

In [ ]:
from math import pi, sqrt
import numpy as np
import numpy.linalg as LA
from scipy import optimize
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Lösung mittels Matrizen

### Berechnung des harmonischen Potentials

Bei der Berechnung des harmonischen Potentials kann das Argument `x` sowohl ein skalarer Wert als auch ein NumPy-Array sein.

In [ ]:
def potential(x):
    return x*x/2

### Berechnung des Hamilton-Operators

Der Hamiltonoperator wird hier in der diskretisierten Form als Matrix erzeugt. Entsprechend dem in der Einleitung angegebenen Ausdruck für die zweite Ableitung ergibt diese von null verschiedene Beiträge in der Diagonalen sowie den benachbarten Nebendiagonalen.

In [ ]:
def hamilton_operator(n_max, x_max):
    ndim = 2*n_max+1
    x, dx = np.linspace(-x_max, x_max, ndim, retstep=True)
    h = (np.eye(ndim)
         - 0.5*(np.eye(ndim, k=1) + np.eye(ndim, k=-1)))
    h = h/dx**2 + np.diag(potential(x))
    return h

### Berechnung der Eigenwerte und Eigenvektoren

Das Eigenwertproblem in Matrixform wird mit Hilfe der für hermitesche Eigenwertprobleme geeigneten Funktion
`LA.eigh` aus der NumPy-Bibliothek gelöst. Die Eigenwerte `evals` sind reell und aufsteigend angeordnet. Die zugehörigen Eigenvektoren befinden sich im zweidimensionalen NumPy-Array `evecs`.

In [ ]:
def eigenproblem(n_max, x_max):
    h = hamilton_operator(n_max, x_max)
    evals, evecs = LA.eigh(h)
    return evals, evecs

### Implementierung der Bedienelemente und graphische Darstellung der Eigenzustände

Mit Hilfe der Schieberegler können die folgenden Parameter eingestellt werden:
- `n_states`: Anzahl der darzustellenden Zustände
- `n_max`: Anzahl der Gitterpunkte auf einer Seite des Ursprungs. Insgesamt werden `2*n_max+1` Gitterpunkte verwendet.
- `x_max`: Ausdehnung des Gitters von `-x_max` bis `x_max`
- `x_plot_max`: Ausdehnung des darzustellenden Bereichs von `-x_plot_max` bis `x_plot_max`

Die Nulllinien der Eigenfunktionen werden um die jeweiligen Eigenenergien verschoben. Zur besseren Interpretation der Ergebnisse wird zusätzlich das harmonische Potential dargestellt.

In [ ]:
widget_dict = {"n_states":
               widgets.IntSlider(
                   value=10, min=1, max=30, step=1,
                   description="Zustände"),
               "n_max":
               widgets.IntSlider(
                   value=200, min=10, max=1000, step=10,
                   description=r"$n_\text{max}$"),
               "x_max":
               widgets.FloatSlider(
                   value=20, min=10, max=100, step=1,
                   description=r"$x_\text{max}$"),
               "x_plot_max":
               widgets.FloatSlider(
                   value=7, min=1, max=10, step=1,
                   description=r"$x_\text{plot,max}$")
               }

@interact(**widget_dict)
def plot_states(n_states, n_max, x_max, x_plot_max):
    evals, evecs = eigenproblem(n_max, x_max)
    x_values = np.linspace(-x_max, x_max, 2*n_max+1)

    fig, ax = plt.subplots()
    for n in range(n_states):
        ax.plot(x_values, 2*evecs[:, n]+evals[n])
    ax.plot(x_values, potential(x_values), color="black")
    ax.set_xlabel("$x$")
    ax.set_ylabel(r"$\psi(x)$")
    ax.set_xlim((-x_plot_max, x_plot_max))
    ax.set_ylim((0, n_states+1))

## Lösung mittels Minimierung des Energieerwartungswerts

### Berechnung des Energieerwartungswerts

Wir betrachten weiterhin das diskretisierte Problem auf einem eindimensionalen Gitter. Somit ist der Zustandsvektor `psi` durch ein eindimensionales NumPy-Array und der Hamilton-Operator durch ein zweidimensionales NumPy-Array gegeben. Der gesuchte Erwartungswert ergibt sich dann im Wesentlichen als zweifaches Matrixprodukt. 

In [ ]:
def energy(psi, hamilton_operator, dx):
    return psi @ hamilton_operator @ psi * dx

### Definition der Randbedingungen für die Optimierung

Die gesuchte diskretisierte Wellenfunktion soll auf 1 normiert sein, so dass

$$\sum_n \vert\psi_n\vert^2 \Delta x = 1$$

gelten muss. Bei der Berechnung des zweiten angeregten Zustands müssen wir zudem die Orthogonalität auf den Grundzustand fordern und definieren dafür die Funktion `scalarproduct`

In [ ]:
def norm_of_psi(psi, dx):
    return LA.norm(psi)**2 * dx - 1

def scalarproduct(psi, phi, dx):
    return psi @ phi * dx

Damit lassen sich nun die Randbedingungen in der Form definieren, die für die Optimierung benötigt wird. Dazu wird jeweils ein Dictionary erzeugt, das zunächst die Art der Randbedingung definiert, in unserem Fall eine Gleichheit. Der Eintrag `fun` gibt die Funktion an, die zur Auswertung der Randbedingung verwendet werden soll, wobei das erste Argument immer die zu optimierende Funktion ist. Weitere Argumente werden mit dem Tupel im Eintrag `args` spezifiziert.

In [ ]:
def normalization(dx):
    return {"type": "eq",
            "fun": norm_of_psi,
            "args": (dx,)}

def orthogonality(wavefunction, dx):
    return {"type": "eq",
            "fun": scalarproduct,
            "args": (wavefunction, dx)}

### Minimierung der Energie

Die Minimierung erfolgt mit Hilfe der Funktion `optimize.minimize` aus dem SciPy-Paket, die neben der Funktion `energy`, die die zu optimierende Größe berechnet, auch den Anfangszustand `initial_state` und die Randbedingungen `constraints` benötigt. Außerdem werden der Hamilton-Operator `hamilton_operator` und die Schrittweite für die Berechnung der Energie und der Randbedingungen benötigt. Der optimierte Zustand `psi` und der zugehörige Energieerwartungswert `e` können mit Hilfe der Attribute `x` bzw. `fun` aus dem Resultatsobjekt `opt_result` erhalten werden.

In [ ]:
def minimize_energy(initial_state, hamilton_operator, dx,
                    constraints):
    opt_result = optimize.minimize(
        energy, initial_state, args=(hamilton_operator, dx),
        constraints=constraints)
    psi = opt_result.x
    e = opt_result.fun
    return psi, e

### Implementierung der Bedienelemente und graphische Darstellung der energetisch niedrigsten drei Eigenzustände

Mit den Schiebereglern lassen sich die folgenden Parameter einstellen:
- `n_max`: Anzahl der Gitterpunkte auf einer Seite des Ursprungs. Insgesamt werden `2*n_max+1` Gitterpunkte verwendet.
- `x_max`: Ausdehnung des Gitters von `-x_max` bis `x_max`

Als Ausgangsfunktion für die symmetrischen Eigenzustände wird

$$\psi_\text{init}= \frac{1}{2\cosh(x/2)}$$

gewählt, während für antisymmetrische Eigenzustände die Funktion

$$\psi_\text{init} = \frac{\sqrt{3}}{2}\frac{x}{\cosh(x/2)}$$

verwendet wird. Beide Zustände sind normiert. Bei der Berechnung des Grundzustands und des ersten angeregten Zustands genügt es, bei der Optimierung sicherzustellen, dass die Normierung erhalten bleibt. Dagegen ist bei der Berechnung des zweiten angeregten Zustands zusätzlich die Orthogonalität zum zuvor berechneten Grundzustand zu fordern. Die Qualität der durch Optimierung erhaltenen Zustände lässt sich anhand der angegebenen Energiewerte abschätzen. Bei einer sehr kleinen Zahl von Gitterpunkten werden Sie feststellen, dass der zweite angeregte Zustand im Rahmen der Optimierung seine Symmetrie wechselt, sofern man nicht die Orthogonalität auf den ersten angeregten Zustand fordert.

In [ ]:
widget_dict = {"n_max":
               widgets.IntSlider(
                   value=100, min=10, max=500, step=10,
                   description=r"$n_\text{max}$"),
               "x_max":
               widgets.FloatSlider(
                   value=10, min=5, max=30, step=1,
                   description=r"$x_\text{max}$"),
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_states_from_variation(n_max, x_max):
    x_values, dx = np.linspace(-x_max, x_max, 2*n_max+1,
                               retstep=True)
    h = hamilton_operator(n_max, x_max)
    initial_state_symm = 1/(2*np.cosh(x_values/2))
    initial_state_antisymm = (sqrt(3)/(2*pi)*x_values
                              / np.cosh(x_values/2))
    psi_0, energy_0 = minimize_energy(
        initial_state_symm, h, dx, normalization(dx))
    psi_1, energy_1 = minimize_energy(
        initial_state_antisymm, h, dx, normalization(dx))
    psi_2, energy_2 = minimize_energy(
        initial_state_symm, h, dx,
        [normalization(dx), orthogonality(psi_0, dx)])

    fig, ax = plt.subplots()
    for psi, energy in ((psi_0, energy_0),
                        (psi_1, energy_1),
                        (psi_2, energy_2)):
        ax.plot(x_values, psi+energy,
                label=f"E={energy:.5f}")
    ax.plot(x_values, potential(x_values), color="black")
    ax.set_xlabel("$x$")
    ax.set_ylabel(r"$\psi(x)$")
    ax.set_xlim((-5, 5))
    ax.set_ylim((0, 5))
    ax.legend(loc="upper left")